# Stazionarietà della serie
## Introduzione ai Test Statistici

Lo studio di serie storiche con modelli statistici (ARMA, ARIMA, SARIMA, Holt-Winters,...) presuppone che la distribuzione abbia alcune caratteristiche come:
* stazionarietà
* correlazione dei dati (i dati sono indipendentemente distribuiti?)
* distibuzione normale dei dati
<br>

Per verificare se sussistono queste caratteristiche bisogna effettuare alcuni test statistici. In python la principale libreria che racchiude questi strumenti è [statsmodels](https://www.statsmodels.org/stable/index.html), in particolare il modulo *tsa*.

## Caricamento di Airpassengers

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Per prima cosa carichiamo la serie e vediamo:
* i primi 5 valori
* il plot dell'originale

In [ ]:
airpassengers = pd.read_csv("airpassenger.csv")
airpassengers.columns = ["Time","Passengers"]
airpassengers.head()

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(range(airpassengers.shape[0]), airpassengers["Passengers"].values, color='tab:red')
plt.gca().set(title="Airpassengers", xlabel="Time", ylabel="Passengers")
plt.show()

## Componenti della serie

Una serie storica ha solitamente quattro componenti:
* *trend*
* *ciclo*
* *stagionalità*
* *componente erratica*
<br>

In tsa esiste una funzione che decompone la serie e ci fa vedere 3 delle quattro componenti sopra elencate.
<br>

**N.B. La differenza tra ciclo e stagionalità. La stagionalità si presenta sui dodici mesi, il ciclo può essere mensile, seetimanale, ... non confondiamo.**

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
decompose = seasonal_decompose(airpassengers["Passengers"], model = "multiplicative", period = 12)

Sapevamo già che la serie è mensile, quindi frequenza 12, il model indica il tipo di stagionalità, se vediamo il grafico i picchi crescono al passare dei periodi, quindi l'interazione tra le componenti è moltiplicativa.
<br>
Nei grafici di seguito vediamo:
1. La serie originale
2. Il trend
3. La stagionalità
4. La componente erratica

In [ ]:
plt.figure(figsize=(16,8))
decompose.plot()
plt.show()

## Test sulla stazionarietà
Quando si valuta la stazionarietà di una serie per prima cosa si fa un'analisi "visuale" delle autocorrelazioni totali e parziali.
<br>
Se la serie è stazionaria avremo un andamento decrescente delle stesse.

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, pacf

### Autocorrelazioni globali

In [ ]:
acf = acf(airpassengers["Passengers"], nlags = 40,qstat= True, fft=False)

Ritorna una tupla di tre array:
* le autocorrelazioni
* la q stat di Box - Jenkins per ogni autocorrelazione
* il p-value associato ad ogni correlazione
<br>
vediamo direttamente il p-value, gli elementi sono quasi pari a zero quindi sotto alpha = 0.05.
<br>

Possiamo concludere che con buona probabilità la serie **non è stazionaria**.

In [ ]:
acf[2]

In [ ]:
plt.figure(figsize=(16,8))
plot_acf(airpassengers["Passengers"], lags = 40)
plt.show()

### Autocorrelazioni parziali

In [ ]:
prt_acf = pacf(airpassengers["Passengers"])

In [ ]:
plt.figure(figsize=(16,8))
plot_pacf(airpassengers["Passengers"])
plt.show()

### ADFuller test
Questo è un test che presenta sotto l'ipotesi nulla:
* **H0**: serie NON stazionaria

In [ ]:
# funzione che confronta il p-value con il livello di significatività del test
# e scrive se accettare o rifiutare l'ipotetsi nulla.
def hypothesis(p_value, alpha):
    if p_value<= alpha:
        print("Non accettiamo H0")
    else:
        print("Accettiamo H0")

In [ ]:
from statsmodels.tsa.stattools import adfuller
adf = adfuller(airpassengers["Passengers"], regression="ct")

In [ ]:
adf

In [ ]:
hypothesis(p_value=round(adf[1],2), alpha=0.05)

Il p-value pari a 0.54 ciò fa si che non possiamo rigettare l'ipotesi nulla, nel nostro caso la NON stazionarietà della serie.

*N.B. I valori vicino a 1 sono detti* ***critici***

### KPSS test
Questo è un test che presenta sotto l'ipotesi nulla:
* **H0**: serie stazionaria

In [ ]:
from statsmodels.tsa.stattools import kpss
kpss_test = kpss(airpassengers["Passengers"], regression = 'ct', nlags = 13, )

**N.B.**
+ **Perchè abbiamo scelto lags = 13?**
<br>
Il default dei lag è 
*(12 * (n / 100) ^ (1 / 4))*
+ **Cos'è il parametro ct?**
<br>
Vogliamo verificare la stazionarietà rispetto al trend

In [ ]:
kpss_test

In [ ]:
hypothesis(round(kpss_test[1],2), alpha=0.05)

Anche questo test ci suggerisce che la serie **NON è** stazionaria.

### Rendere la serie stazionaria
Per rendere la serie stazionaria, visto il trend lineare, dovrebbe essere sufficiente calcolare la differenziazione prima che dovrebbe mitigare se non eliminare l'effetto trend.
<br>
Per avere conferma dell'efficacia della differenziazione ripeteremo i test sulla serie differenziata.

In [ ]:
stat = airpassengers["Passengers"].diff()

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(stat)
plt.title("Airpassengers [stazionaria?]")

In [ ]:
adf_stat = adfuller(stat.values[1:])
print("p-value: ", round(adf_stat[1], 2))
hypothesis(p_value=round(adf_stat[1], 2), alpha=0.05)

In [ ]:
kpss_stat = kpss(stat.values[1:], nlags="legacy")
print("p-value: ", round(kpss_stat[1], 2))
hypothesis(p_value=round(kpss_stat[1], 2), alpha=0.05)

Possiamo affermare che dopo la differenziazione prima la serie **è stazionaria**
<br>
**N.B. avrete notato che arrotondo il p-value a due cifre, questo per non essere portato a rigettare H0 per uno scarto minimo. Potrebbe essere un errore.**

Per quanto riguarda la distribuzione dei dati vi lascio il nome dei test che potete provare a fare da soli e giungere alle  conclusioni su Airpassengers:
* indipendenza: *Ljung-Box*
* normalità: *Jarque-Bera*